In [1]:
# this block is the initial class with SABR and blackScholes vol analytical formulae

import scipy as sc
import numpy as np
import pandas as pd
import scipy.stats as stats
import math
import datetime
from datetime import datetime
from scipy import optimize
from scipy.optimize import minimize
import time
import json
from dateutil import parser
from dateutil.parser import parse
import matplotlib.pyplot as plt
import websocket
from websocket import create_connection

import abc
from abc import ABCMeta, abstractmethod

try:
    import pysabr
except:
    !pip install pysabr
    import pysabr

try:
    
    import websockets
except:
    !pip install websockets
    import websockets


print('end loading of modules')

end loading of modules


In [12]:
# the below are the requirements for NFT bot. Either put in requirements.txt and run it as
# pip install -r requirements.txt; OR import the modules in the jupyter notebook

# prawcore==2.3.0
# psaw==0.1.0
# requests==2.26.0
# update-checker==0.18.0
# urllib3==1.26.6
# websocket-client==1.2.1

import prawcore
import requests
import update_checker
import urllib3
import websockets

from psaw import PushshiftAPI
import praw
import random
import time
import configparser
import re
import webbrowser
import warnings
warnings.filterwarnings("ignore")

In [28]:
# Parse Config
# detailed instructions to create the reddit info below
# first, create an Reddit account (duh!) at reddit.com
# create a new reddit 'app' using this link (make sure you are logged in) : https://www.reddit.com/prefs/apps
# enter the necessary details in the config_NFTbot.ini but note the comments of where to get the fields below

cfg = configparser.RawConfigParser()
cfg.read("C:\\config_NFTbot.ini");

# Client_ID is the 'personal use script' ID. It will also be emailed to you after creation
API_REDDIT_CLIENT_ID = eval(cfg["Credentials"]["API_REDDIT_CLIENT_ID"])
# Client_Secret is the ID shown after 'secret'
API_REDDIT_CLIENT_SECRET = eval(cfg["Credentials"]["API_REDDIT_CLIENT_SECRET"])
# Username is your login username to your account
API_REDDIT_USERNAME = eval(cfg["Credentials"]["API_REDDIT_USERNAME"])
# this is the name you have given your application
API_REDDIT_USER_AGENT = eval(cfg["Credentials"]["API_REDDIT_USER_AGENT"])
# this is your Reddit login password
API_REDDIT_PASSWORD = eval(cfg["Credentials"]["API_REDDIT_PASSWORD"])

# this is your DeFi onchain wallet address (usually ethereum)
OPENSEA_WALLET = eval(cfg["Bot"]["OPENSEA_WALLET"])
MAX_GIVEAWAYS = eval(cfg["Bot"]["MAX_GIVEAWAYS"])
MIN_SECS_SLEEP = eval(cfg["Bot"]["MIN_SECS_SLEEP"])
MAX_SECS_SLEEP = eval(cfg["Bot"]["MAX_SECS_SLEEP"])

REDDIT_SUBS = eval(cfg["Reddit"]["REDDIT_SUBS"])
REDDIT_COMMENTS = eval(cfg["Reddit"]["REDDIT_COMMENTS"])
REDDIT_EMOJIS = eval(cfg["Reddit"]["REDDIT_EMOJIS"])

In [30]:
praw_api = praw.Reddit(
    client_id=API_REDDIT_CLIENT_ID,
    client_secret=API_REDDIT_CLIENT_SECRET,
    username=API_REDDIT_USERNAME,
    user_agent=API_REDDIT_USER_AGENT,
    password=API_REDDIT_PASSWORD,
)

l_post_author = []
psaw_api = PushshiftAPI()
submissions = psaw_api.search_submissions(
    subreddit=REDDIT_SUBS,
    q="GIVEAWAY|Giveaway|giveaway",
    filter=["id"],
)

n_giveaways_found = 1
while True:
    try:
        submission = next(submissions, None)

        # Check if search_submissions didn't get anymore posts
        if not submission:
            break

        # Get more information about post using PRAW api
        submission = praw_api.submission(id=submission.id)

        # Ensure that the post hasn't been removed  by moderator in the meanwhile,
        # that there is a description and it's not just an image, and that we aren't 
        # re-considering same author's watchlist
        if (
            not submission.removed_by_category
            and submission.selftext
            and submission.author.name not in l_post_author
        ):
            text_from_op = submission.selftext
            
            have_seen_post_before = False
            for comment in submission.comments:
                # The bot has already commented here, so skip this post
                if comment.author.name == API_REDDIT_USERNAME:
                    have_seen_post_before = True
                # Comment from author it may contain the opensea url we are interested
                elif comment.author.name == submission.author.name:
                    text_from_op += comment.body
                    
            # If we have see this post before, just skip it
            if have_seen_post_before:
                continue
                
            # If there's no reference to opensea, the giveaway may be in another chain
            # If the subreddit doesnt contain opensea name, then we are confident that the
            # the collection/item is in another chain. Thus, we skip this post
            if "opensea" not in text_from_op.lower() and "opensea" not in submission.subreddit.display_name.lower():
                continue
            
            print("URL: ", submission.url)
            print("TITLE: ", submission.title)
            
            # Upvote post
            submission.upvote()
            
            # Comment on post
            comment = REDDIT_COMMENTS[random.randint(0,len(REDDIT_COMMENTS)-1)]
            emoji = REDDIT_EMOJIS[random.randint(0,len(REDDIT_EMOJIS)-1)]
            submission.reply(f"{comment} {OPENSEA_WALLET} {emoji}")
            
            # Extract opensea url
            try:
                opensea_url = re.search("(?Phttps?://opensea.io[^\s]+)", text_from_op).group("url")
                if opensea_url:
                    if ']' in opensea_url:
                        opensea_url = opensea_url.split("]")[0]
                    if ')' in opensea_url:
                        opensea_url = opensea_url.split(")")[0]
                    print("OPENSEA: ", opensea_url)
                    webbrowser.open(opensea_url)
            except:
                print("No Opensea URL")
            
            # Wait a random amount of time to avoid getting recognized as a bot
            secs_to_wait = random.randint(MIN_SECS_SLEEP, MAX_SECS_SLEEP)
            time.sleep(secs_to_wait)

            # Increment count of valid NFT giveaways posts found
            n_giveaways_found += 1
            
            # This avoids to comment on posts from same author across multiple subs
            l_post_author.append(submission.author.name)
            print("")

        # Check if number of wanted posts found has been reached
        if n_giveaways_found > MAX_GIVEAWAYS:
            break
            
    except:
        print(
            "Error occurred!"
        )

print("The end.")

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


Exception: Unable to connect to pushshift.io. Max retries exceeded.